## Import dependencies

In [91]:
import sys, os
from graph_builder import *
import pandas as pd
from copy import deepcopy
import csv

 ## Build all graphs from the available tables

In [92]:
sys.path.append('C:\\Users\\ronal\\Documents\\GitHub\\decision-making\\graph_builder') # Append path where graph_builder.py is saved

file_names = os.listdir('treebuilderUpdated')
graphs = {}
for file in file_names:
    graphs[file.replace('.csv', '')] = build_graph('treebuilderUpdated\\' + file)


 # We begin to write the path analysis program

 ## Helper functions

In [93]:
# Helper function for analyze_path()
def parse_path_string(path_string):
    a = path_string.replace('p','')
    b = a.split(';')
    b.remove('')
#     e = [tuple(int(s) for s in i.split(',')) for i in d]         Useful for converting strings of tuples in real tuples
    
    return b



In [94]:
# Helper function for analyze_path(). Returns the root node for a graph.
def root_node(graph):
    root = [v for v, d in graph.in_degree() if d == 0][0]
    return root



In [95]:
def leaf_nodes(graph):
    leaves = [v for v, d in graph.out_degree() if d == 0]
    return leaves



In [96]:
# Helper function for analyze_path()
def value_calculation(graph, node, steps_so_far, prior_prob):
    new_observations = graph.nodes[node]['new_observations']
    ep = graph.nodes[node]['node_ep']
    value = ((steps_so_far * new_observations) + ep) * prior_prob
    
    return value



In [97]:
# Helper function for analyze_path(). Returns the node in successors which corresponds to the input_location if it is found. Returns False otherwise.
def location_in_successors(graph, input_location, current_node):
    node_location_dict = (nx.get_node_attributes(graph, 'node_location'))
    successor_list = list(graph.successors(current_node))
    
    for successor in successor_list:
        if node_location_dict[successor] == input_location:
            return successor
        
    else:
        return False



In [98]:
def num_successors(graph, node):
    num_of_successors = len(list(graph.successors(node)))
    
    return num_of_successors



In [99]:
def get_successors(graph, node, index=None): # Returns list of successors. If given an index, returns the successor with that index in the list.
    if index is None:
        return list(graph.successors(node))
    
    else:
        return list(graph.successors(node))[index]



In [100]:
def all_node_paths(graph):
    all_paths = []
    
    for path in nx.all_simple_paths(graph, root_node(graph), leaf_nodes(graph)):
        all_paths.append(path)
        
    return all_paths



In [101]:
def possible_subject_paths(graph, subject_sequence): # Input list of nodes subject visited. Returns possible paths that subject could have taken to reach leaf nodes.
                                            # Use if last node in subject_sequence is not in leaf_nodes().
    subject_set = set(subject_sequence)
    num_common_nodes = [] # Number of nodes that each sequence in all_node_paths() has in common with subject_sequence
    for path in all_node_paths(graph):
        num_common_nodes.append(len(set(path) & subject_set))

#     print(num_common_nodes)

    similarity_degree = max(num_common_nodes)

    possible_subject_paths = [path for idx, path in enumerate(all_node_paths(graph)) if num_common_nodes[idx] == similarity_degree]
    
    
    return possible_subject_paths



In [102]:
def generic_path_value(graph, path): # Returns (generic) value of the node path (as defined by the last node in the path).
    last_node = path[-1]
    
    generic_path_value = graph.nodes[last_node]['node_value']
    
    return generic_path_value
    



In [103]:
def extra_node_lists(graph, subject_sequence): # Input list of nodes subject visited. Returns list of sequences of remaining nodes for each possible subject path.
    _possible_subject_paths = possible_subject_paths(graph, subject_sequence)
    num_possible_paths = len(_possible_subject_paths)
    extra_node_lists = [[node for node in _possible_subject_paths[i] if node not in subject_sequence] for i in range(num_possible_paths)]
    
    return extra_node_lists



In [104]:
def prior_prob(graph):
    _root_node = root_node(graph)
    total_black_squares = graph.nodes[_root_node]['black_remains']
    prior_prob = 1/total_black_squares
    
    return prior_prob



In [105]:
def valid_location(graph, step, visited_locations, current_node, previous_node):
    if previous_node == '':
        return
    
    successors = get_successors(graph, current_node)
    previous_successors = get_successors(graph, previous_node)
    
    node_location_dict = nx.get_node_attributes(graph, 'node_location')
    
    successor_locations = [node_location_dict[k] for k in node_location_dict if k in successors] # Successor locations for current node
    previous_successor_locations = [node_location_dict[k] for k in node_location_dict if k in previous_successors] # Successor locations for previous node
    
    node_locations = list(node_location_dict.values())
    
    valid_locations = visited_locations.copy()

    valid_locations.update(successor_locations)
    valid_locations.update(previous_successor_locations)
    
    
    if step in node_locations:
        if step in valid_locations:
            pass
        else:
            return False



In [106]:
def alt_node_paths(graph, chosen_path):
    _all_node_paths = all_node_paths(graph)
    alt_node_paths = _all_node_paths.copy()
    alt_node_paths.remove(chosen_path)
    
    path_value_dict = {','.join(path):generic_path_value(graph, path) for path in alt_node_paths} # Keys are a string of nodes seperated by commas. Use list.split() method to convert into list.
    
    return path_value_dict
    
    


 ## Path analysis function

In [124]:
def analyze_path(graph, path): # Takes path string as input.
    PRIOR_PROB = prior_prob(graph)
    steps_so_far = 0
    
    current_node_value = 0
    current_node = root_node(graph)
    previous_node = ''
    node_sequence = [root_node(graph)]
    path_value = 0
    input_path = parse_path_string(path)
    LEAF_NODES = leaf_nodes(graph)
    subject_graph = deepcopy(graph)
    
    all_visited_locations = {'(0,0)'} # First visited location is subject's starting position.

    for step in input_path[1:]:
        if valid_location(subject_graph, step, all_visited_locations, current_node, previous_node) is False:
            return ['ERROR_PATH', step, current_node]
        else:
            if location_in_successors(subject_graph, step, current_node) is not False:
                steps_so_far += 1
                
                previous_node = current_node
                current_node = location_in_successors(subject_graph, step, current_node)
                node_sequence.append(current_node)
                
                path_value += value_calculation(subject_graph, current_node, steps_so_far, PRIOR_PROB)
                current_node_value = path_value
                subject_graph.nodes[current_node]['node_value'] = current_node_value
                
            else:
                steps_so_far += 1
            all_visited_locations.add(step)
    
    empirical_path_value = path_value # Saves path value at the last node the subject visited in experiment
    empirical_path = node_sequence # Saves sequence of nodes subject visited in experiment
    empirical_last_node = current_node
    
#     print('empirical path is', empirical_path)
#     print('empirical step number is', steps_so_far)
#     print('empirical pathvalue is', empirical_path_value)
#     print('empirical last node is', empirical_last_node)
    
#     #####################################################################################
    _extra_node_lists = extra_node_lists(subject_graph, empirical_path)
    path_comparison_dict = {}
    
    for node_list in _extra_node_lists:
        node_sequence = empirical_path
        path_value = empirical_path_value
        current_node = empirical_last_node
        
        while True:
            try: 
                next_node = node_list.pop(0)
                steps_so_far += subject_graph.succ[current_node][next_node]['steps_from_parent']
                
                current_node = next_node
                node_sequence.append(current_node)
                path_value += value_calculation(subject_graph, current_node, steps_so_far, PRIOR_PROB)
                current_node_value = path_value
                subject_graph.nodes[current_node]['node_value'] = current_node_value

            except IndexError: # Stop while loop when we have reached the last node in the current node list
                path_comparison_dict[','.join(node_sequence)] = path_value
                break
                
    final_path_string = min(path_comparison_dict, key = lambda k: path_comparison_dict[k]) # If the subject did not reach a leaf, this is the path we assume the subject would have taken (the optimal choice).
    final_path = final_path_string.split(',')
    final_path_value = path_comparison_dict[final_path_string]
    
    alt_path_values = sorted(list(
        alt_node_paths(subject_graph, final_path).values()
                                ))
    
    all_values = [str(round(final_path_value, 3))]
    all_values.extend([str(round(num, 3)) for num in alt_path_values])

    output = [final_path_string, round(final_path_value, 3)]
    output.append(';'.join(all_values))
    
#     return output
    
#   print(node_sequence)
    print(path_comparison_dict)
    


In [125]:
analyze_path(graphs['library'], 'p(0,0);p(0,1);p(1,1);p(1,2);p(1,3);p(1,4);p(1,3);p(2,3);p(3,3);p(4,3);p(5,3);p(6,3);p(6,4);')

{'N3151,N8688,N1452,N8106,N3650,N4693,N4422,N6940': 21.5, 'N3151,N8688,N1452,N8106,N3650,N4693,N4422,N6940,N8688,N1452,N8106,N3650,N4693,N500,N4478': 36.66666666666667, 'N3151,N8688,N1452,N8106,N3650,N4693,N4422,N6940,N8688,N1452,N8106,N3650,N4693,N500,N4478,N8688,N1452,N54,N4714,N3570,N8310,N5359': 52.74999999999999, 'N3151,N8688,N1452,N8106,N3650,N4693,N4422,N6940,N8688,N1452,N8106,N3650,N4693,N500,N4478,N8688,N1452,N54,N4714,N3570,N8310,N5359,N8688,N1452,N54,N4714,N3570,N3015,N7821': 74.41666666666667, 'N3151,N8688,N1452,N8106,N3650,N4693,N4422,N6940,N8688,N1452,N8106,N3650,N4693,N500,N4478,N8688,N1452,N54,N4714,N3570,N8310,N5359,N8688,N1452,N54,N4714,N3570,N3015,N7821,N8688,N1452,N54,N4714,N3348,N8906,N6836': 98.91666666666664, 'N3151,N8688,N1452,N8106,N3650,N4693,N4422,N6940,N8688,N1452,N8106,N3650,N4693,N500,N4478,N8688,N1452,N54,N4714,N3570,N8310,N5359,N8688,N1452,N54,N4714,N3570,N3015,N7821,N8688,N1452,N54,N4714,N3348,N8906,N6836,N8688,N1452,N54,N4714,N3348,N9384,N3970': 120.74

 ## Analyze experimental data

In [108]:
def parse_data(data_file):
    experiment_data_frame = pd.read_csv(data_file, sep='\t')

    previous_subject = ''
    previous_world = ''
    previous_path = ''
    # previous_index = 0
    parsed_data = []

    # i = 0
    for row in experiment_data_frame.itertuples():
        if previous_subject != row[1]:
            parsed_data.append([previous_subject, previous_world, previous_path])

    #     previous_index = index + 2
        previous_subject = row[1]
        previous_world = row[2]
        previous_path = row[3]

    parsed_data.append( # Add last row of the data frame manually, since algorithm above misses it
                      [experiment_data_frame.iloc[-1]['subject'], 
                       experiment_data_frame.iloc[-1]['world'], 
                       experiment_data_frame.iloc[-1]['squarepath']
                      ]
                     )    

    #     if i == 2000:
    #         break
    #     i += 1

    parsed_data.pop(0)

    return parsed_data



In [109]:
parsed_data = parse_data('SquareLabelsWithRT_E2.csv')

In [110]:
def analyze_data(parsed_data):
    num_error_paths = 0
    output_data = deepcopy(parsed_data)

    for row in output_data: # Analyze path that each subject followed
        graph_name = row[1]
        input_path = row[2]
        path_analysis = analyze_path(graphs[graph_name], input_path)
        row.extend(path_analysis)

        if path_analysis[0] == 'ERROR_PATH':
            num_error_paths += 1

    print(f'{len(output_data)} paths analyzed')
    print(f'There are {num_error_paths} error paths')
    
    return output_data



In [111]:
output_data = analyze_data(parsed_data)

1512 paths analyzed
There are 45 error paths


In [112]:
# Export path analysis as csv
def export_results(output_data):
    column_titles = ['subject', 'world', 'square_path', 'chosen_node_path', 'chosen_value', 'all_values'] # The first item in the column 'all_values' is the chosen value

    with open('anaylzed_subject_data.csv', 'w') as file:
        file_writer = csv.writer(file, delimiter='\t')
        file_writer.writerow(column_titles)

        for row in output_data:
            file_writer.writerow(row)



In [113]:
export_results(output_data)

In [114]:
# Export error data
def export_error_data(output_data):
    error_data = []
    for row in output_data:
        if row[3] == 'ERROR_PATH':
            error_data.append(row)

    column_titles = ['subject', 'world', 'square_path', 'path_type', 'error_step', 'error_node']

    with open('error_data.csv', 'w') as file:
        file_writer = csv.writer(file, delimiter='\t')
        file_writer.writerow(column_titles)

        for row in error_data:
            file_writer.writerow(row)



In [115]:
export_error_data(output_data)

## Update graph nodes with successor values

In [46]:
nx.get_node_attributes(graphs['courtyard'], 'node_value')

{'N6609': 0.0,
 'N2554': 2.2,
 'N2200': 3.5,
 'N6588': 3.4,
 'N5553': 7.3999996,
 'N6173': 6.2,
 'N7822': 6.8,
 'N3405': 10.599999,
 'N5323': 8.2}

In [47]:
for node in graphs[graph].nodes():
    

N6609
N2554
N2200
N6588
N5553
N6173
N7822
N3405
N5323
